In [5]:
from pathlib import Path 
import os
import torch
import pandas as pd
from tqdm import tqdm
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
from napatrackmater.Trackvector import (
    inception_model_prediction,
    save_cell_type_predictions,
    SHAPE_DYNAMIC_FEATURES
)
from sklearn.decomposition import PCA
from matplotlib import cm
import matplotlib.pyplot as plt
from ipywidgets import interact, Dropdown, IntSlider

In [10]:
dataset_name = 'Sixth'
home_folder = '/home/debian/jz/'
channel = 'nuclei_'
tracking_directory = f'{home_folder}Mari_Data_Oneat/Mari_{dataset_name}_Dataset_Analysis/nuclei_membrane_tracking/'
data_frames_dir = os.path.join(tracking_directory, f'dataframes/')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
tracklet_length = 25
normalized_dataframe = os.path.join(data_frames_dir , f'goblet_basal_dataframe_normalized_{channel}.csv')
print(f'reading data from {normalized_dataframe}')
tracks_dataframe = pd.read_csv(normalized_dataframe)
n_components = 3

class_map_gbr = {
        0: "Basal",
        1: "Radial",
        2: "Goblet"
    }

reading data from /home/debian/jz/Mari_Data_Oneat/Mari_Sixth_Dataset_Analysis/nuclei_membrane_tracking/dataframes/goblet_basal_dataframe_normalized_nuclei_.csv


In [15]:
all_trackmate_ids = [trackid  for trackid in tracks_dataframe['TrackMate Track ID'].unique()]
result_dict = {cell_type: {} for cell_type in class_map_gbr.values()}

unique_time_points = tracks_dataframe['t'].unique()

for time_point in unique_time_points:
        time_data = tracks_dataframe[tracks_dataframe['t'] == time_point]

        for cell_type in class_map_gbr.values():
            cell_type_data = tracks_dataframe[tracks_dataframe['Cell_Type'] == cell_type]

            if not cell_type_data.empty:
                
                features_list = []
                for track_id in cell_type_data['Track ID'].unique():
                    track_features = cell_type_data[cell_type_data['Track ID'] == track_id][SHAPE_DYNAMIC_FEATURES].to_numpy()
                    features_list.append(track_features)

                concatenated_features = np.vstack(features_list)
                
                if concatenated_features.shape[0] > n_components:  

                   
                    pca = PCA(n_components=n_components)
                    principal_components = pca.fit_transform(concatenated_features)

                    if time_point not in result_dict[cell_type]:
                        result_dict[cell_type][time_point] = []

                    result_dict[cell_type][time_point].append(principal_components)


In [13]:

def plot_pcs(result_dict, cell_type, time_point):
    if cell_type in result_dict and time_point in result_dict[cell_type]:
        pcs = np.vstack(result_dict[cell_type][time_point])

        plt.figure(figsize=(12, 6))

        # PC1 vs PC2
        plt.subplot(1, 2, 1)
        plt.scatter(pcs[:, 0], pcs[:, 1], alpha=0.7)
        plt.xlabel('PC1')
        plt.ylabel('PC2')
        plt.title(f'PC1 vs PC2 for {cell_type} at time {time_point}')

        # PC2 vs PC3
        plt.subplot(1, 2, 2)
        plt.scatter(pcs[:, 1], pcs[:, 2], alpha=0.7)
        plt.xlabel('PC2')
        plt.ylabel('PC3')
        plt.title(f'PC2 vs PC3 for {cell_type} at time {time_point}')

        plt.tight_layout()
        plt.show()
    else:
        print(f"No data available for {cell_type} at time {time_point}")

# Interactive widget setup
def interactive_plot(result_dict, class_map_gbr, unique_time_points):
    interact(
        plot_pcs,
        result_dict=widgets.fixed(result_dict),
        cell_type=Dropdown(options=list(class_map_gbr.values()), description='Cell Type:'),
        time_point=IntSlider(min=min(unique_time_points), max=max(unique_time_points), step=1, description='Time Point:')
    )

{'Basal': {}, 'Radial': {}, 'Goblet': {}}

In [ ]:
interactive_plot(result_dict, class_map_gbr, tracks_dataframe['t'].unique())